In [ ]:
# |default_exp chroma_db

## Install dependencies

## Make an app with Gradio

In [ ]:
# |export
import csv
import re
import chromadb
from chromadb import Settings
import pandas as pd
from dotenv import load_dotenv
import os

import gradio as gr
# from fastcore.net import urljson, HTTPError
# from openai import api_key
# from openai import OpenAI

In [ ]:
# |export
# import chromadb.utils.embedding_functions as embedding_functions
# from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [ ]:
# load_dotenv()
# gemini_key = os.getenv('GEMINI_API_KEY')
# deepseek_key = os.getenv('DEEPSEEK_R1_API_KEY')

In [ ]:
print(os.environ.get('HTTPS_PROXY'))


In [ ]:
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key=deepseek_key,
# )

In [ ]:
# deepseek_ef = OpenAIEmbeddingFunction(
#     api_key=deepseek_key,
#     api_base="https://openrouter.ai/api/v1",
#     model_name="deepseek_r1：free",
# )

In [ ]:

# completion = client.chat.completions.create(
#   extra_headers={
#     "HTTP-Referer": "binjian.github.io", # Optional. Site URL for rankings on openrouter.ai.
#     "X-Title": "binjian's digital garden", # Optional. Site title for rankings on openrouter.ai.
#   },
#   extra_body={},
#   model="deepseek/deepseek-r1:free",
#   messages=[
#     {
#       "role": "user",
#       "content": "What's Anthropic's/meb Model context protocol?"
#     }
#   ]
# )
# print(completion.choices[0].message.content)

In [ ]:
# google_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=gemini_key)

In [ ]:
def convert_qa_to_csv(input_file, output_file):
    """
    Convert a text file with Q/A format to a CSV file.

    Args:
        input_file: Path to the input text file
        output_file: Path to the output CSV file
    """
    # Read the content of the file
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the content by 'Q' marker
    qa_blocks = content.split('Q\n')

    qa_blks = [block.strip() for block in qa_blocks][1:]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [[line for line in block.split('\n') ] for block in qa_blks if block.strip()]
    # Remove empty blocks (like the first one if file starts with 'Q')
    # qa_blocks = [blk for block in qa_blocks if block.strip() for blk in block.strip()]

    # Process each Q&A block
    qa_pairs = []
    for block in qa_blks:
        # Split the block into lines
        lines = block.strip().split('\n')

        if lines:
            # First line is the question
            question = lines[0]
            # The rest are the answer
            answer = '\n'.join(lines[1:])

            # Add the pair to our list
            qa_pairs.append([question, answer])

    # Write to CSV
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        # Write header
        writer.writerow(['Question', 'Answer'])
        # Write Q&A pairs
        for pair in qa_pairs:
            writer.writerow(pair)
    print(f"Conversion complete. CSV file saved to {output_file}")

In [ ]:

input_files = ["../res/qa_service.txt", "../res/qa_technology.txt"]
output_files = ["../res/qa_service.csv", "../res/qa_technology.csv"]
for in_f, ot_f in zip(input_files, output_files):
    convert_qa_to_csv(in_f, ot_f)
    

In [ ]:
# |export
client = chromadb.PersistentClient(path="../db")
collections = [client.get_or_create_collection(name="siasun_qa_service"),
               client.get_or_create_collection(name="siasun_qa_technology")]
# collections = [client.get_or_create_collection(name="siasun_qa_service",embedding_function=deepseek_ef),
#                 client.get_or_create_collection(name="siasun_qa_technology",embedding_function=deepseek_ef)]
# collections = [client.get_or_create_collection(name="siasun_qa_service", embedding_function=google_ef),
#                client.get_or_create_collection(name="siasun_qa_technology", embedding_function=google_ef)]

In [ ]:
# |export
for csv_file,collection in zip(output_files,collections):
    with open(csv_file, newline='') as f:
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            collection.add(
                documents = row,
                metadatas = [{"source": "question"}, {"source": "answer"}],
                ids = [f"{collection.name}_q{i}", f"{collection.name}_a{i}"]
            )

In [ ]:
#|export
queries=["你们的产品需要多久维护一次?","我怎么设置机器人的安全工作区域?"]
queries[0]

In [ ]:
results = collections[0].query(
    query_texts=queries,
    n_results=4
)

In [ ]:
results

In [ ]:
# results['metadatas'][0] #[0]['source']
results['documents'][0][0]

In [ ]:
colls = client.list_collections()
colls

In [ ]:
queries1 = ['你们的产品需要多久维护一次?','宏作业有什么用?']

In [ ]:
def qa(questions:list[str], collections:list[chromadb.Collection]=collections):
    # Check the encoding of input question to select appropriate embedding model

    matched_questions = []
    for collection in collections:
        results = collection.query(
            query_texts=questions,
            n_results=4
        )
        docs = []
        for i,metadata in enumerate(results['metadatas'][0]):
            if metadata['source'] == 'question':
                docs.append({'id': results['ids'][0][i],
                                'document': results['documents'][0][i], 
                                'distance':results['distances'][0][i]})
        df = pd.DataFrame(docs)
        matched_questions.append(df)

    df_matched_questions = pd.concat(matched_questions,axis=0,ignore_index=True)
    best_match_q_id = df_matched_questions.loc[df_matched_questions['distance'].idxmin()]['id']
    id_a_list = best_match_q_id.split('_')
    id_a_list[-1] = id_a_list[-1].replace('q','a')
    id_a = '_'.join(id_a_list)
    coll_idx = 0 if id_a_list[-2] == 'service' else 1
    best_answer = collections[coll_idx].get(id_a)['documents']
    res_text = best_answer[0]
    return res_text
    # question =
    # return answers

In [ ]:

question = "你们的产品需要多久维护一次?"
res = qa(question)
res


In [ ]:
print(res)

In [ ]:
# |export
iface = gr.Interface(fn=qa, inputs=gr.Text(value="多久维护一次产品?"), outputs="text")
iface.launch(width=500,share=True)

In [ ]:
# this is only necessary in a notebook
iface.close()

## Create a `requirements.txt` file

In [ ]:
%%writefile ../requirements.txt
fastcore

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

In [ ]:
# |default_exp data_preprocessing

## Convert this notebook into a Gradio app

In [ ]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()